In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import seaborn as sns
import lightgbm as lgb
import pandas as pd
# 导入必要的Python库
# tqdm: 用于显示进度条，让长时间运行的代码看起来更友好
# matplotlib.pyplot: 用于数据可视化，创建各种图表
# seaborn: 基于matplotlib的高级可视化库，提供更美观的图表
# lightgbm: 微软开发的梯度提升框架，用于机器学习建模
# pandas: 数据处理和分析的核心库，用于处理表格数据

In [10]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, log_loss
# 导入scikit-learn库中的关键模块
# StandardScaler: 标准化处理器，用于将特征缩放到均值为0，方差为1的标准正态分布
# LabelEncoder: 标签编码器，将类别特征转换为数值标签
# KFold: K折交叉验证工具，用于模型验证和防止过拟合
# roc_auc_score: ROC曲线下面积指标，用于评估二分类模型的性能
# log_loss: 对数损失函数，用于评估概率预测的准确性

In [ ]:
train_df=pd.read_csv("/kaggle/input/deepshare-playground-series2/train_data.csv")
test_df=pd.read_csv("/kaggle/input/deepshare-playground-series2/test_data.csv")
# 从Kaggle数据集路径读取训练和测试数据
# train_df: 训练数据集，包含特征和标签，用于训练机器学习模型
# test_df: 测试数据集，通常只包含特征，用于模型预测
# 注意：需要先在Kaggle上下载数据集并放在指定路径下才能运行
# 数据集路径：/kaggle/input/deepshare-playground-series2/

In [ ]:
df=pd.concat([train_df,test_df],axis=0).reset_index(drop=True)
# 合并训练集和测试集数据
# pd.concat(): pandas的合并函数，将多个数据框按行或列合并
# axis=0: 表示按行合并（垂直堆叠）
# reset_index(drop=True): 重新设置索引，删除原来的索引
# 这样做的目的通常是为了统一处理特征工程，保持一致性

In [ ]:
num_cols=[f'user_fea{i}' for i in range(6,9)]
cate_cols=['user_id','item_id']+[f'user_fea{i}' for i in range(1,6)]+['item_fea1']+['user_fea9']
# 定义数值特征和类别特征列名
# num_cols: 数值特征列，包含user_fea6, user_fea7, user_fea8
# 这些是连续的数值型特征，通常需要进行标准化处理
# cate_cols: 类别特征列，包含用户ID、物品ID和其他分类特征

In [ ]:
#连续特征
for col in tqdm(num_cols):
    plt.figure(dpi=150)
    sns.displot(df[col])
# 可视化数值特征的分布情况
# 这有助于了解数据的分布特征，发现异常值或偏态分布
# tqdm(num_cols): 为循环添加进度条，显示处理进度
# plt.figure(dpi=150): 设置图表分辨率为150dpi，提高图像清晰度
# sns.displot(): seaborn的分布图函数，显示数据的分布情况
# 这个循环会为每个数值特征生成一个分布图

In [ ]:
# 正确的标准化流程：只在训练集上拟合，然后转换训练集和测试集
scaler=StandardScaler()
# 第一步：在训练集上拟合scaler（计算均值和标准差）
scaler.fit(train_df[num_cols])
# 第二步：用同一个scaler转换训练集
train_df[num_cols]=scaler.transform(train_df[num_cols])
# 第三步：用同一个scaler转换测试集
test_df[num_cols]=scaler.transform(test_df[num_cols])
# 重要原则：
# 1. 标准化参数（均值、标准差）只能从训练集学习
# 2. 测试集必须使用训练集学习到的参数进行转换
# 3. 这样可以避免数据泄露，确保模型评估的公正性
# 4. 绝不能在合并数据上fit，否则会造成信息泄露

In [ ]:
# 正确的类别特征编码流程：只在训练集上拟合，然后转换训练集和测试集
for col in tqdm(cate_cols):
    le=LabelEncoder()
    # 第一步：在训练集上拟合LabelEncoder
    le.fit(train_df[col])
    # 第二步：用同一个LabelEncoder转换训练集
    train_df[col]=le.transform(train_df[col])
    # 第三步：用同一个LabelEncoder转换测试集
    test_df[col]=le.transform(test_df[col])
# 对类别特征进行标签编码
# LabelEncoder: 将类别特征转换为数值标签（0, 1, 2, ...）
# 例如：['男', '女', '男'] -> [0, 1, 0]
# 重要原则：
# 1. 编码映射关系只能从训练集学习
# 2. 测试集必须使用训练集学习到的映射关系
# 3. 这样可以避免数据泄露，确保模型评估的公正性
# 4. 如果测试集有训练集中未出现的类别，需要特殊处理

In [ ]:
train_df=df[df['label'].notna()].reset_index(drop=True)
test_df=df[df['label'].isna()].reset_index(drop=True)
# 根据标签列重新分割训练集和测试集
# df['label'].notna(): 选择标签不为空的行（训练数据）
# df['label'].isna(): 选择标签为空的行（测试数据）
# reset_index(drop=True): 重新设置索引，删除原来的索引
# 这种分割方式假设原始数据中训练集有标签，测试集没有标签

In [ ]:
train_df.shape,test_df.shape
# 查看训练集和测试集的维度（形状）
# .shape 返回一个元组，表示数据的行数和列数
# 例如：(1000, 20) 表示1000行20列的数据
# 这有助于确认数据分割是否正确，以及数据规模

In [ ]:
drop_feature=['ID','label','day','item_desc','lng','lat','ts','user_tag']
features=[x for x in train_df.columns if x not in drop_feature]
print(len(features),features)
# 选择用于建模的特征
# drop_feature: 需要删除的列，这些列通常不是模型输入特征
# ID: 样本ID，没有预测价值
# label: 标签列，这是我们要预测的目标
# day, item_desc, lng, lat, ts, user_tag: 可能需要特殊处理或暂时不用的特征
# features: 最终用于训练模型的特征列表
# len(features): 特征的数量

In [ ]:
#模型调用
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(train_df[features],train_df['label'],test_size=0.2,random_state=2025)
# 划分训练集和验证集
# train_test_split: 将训练数据进一步分为训练集和验证集
# train_df[features]: 特征数据
# train_df['label']: 标签数据
# test_size=0.2: 验证集占20%，训练集占80%
# random_state=2025: 随机种子，确保每次划分结果一致
# 返回四个数组：训练特征、验证特征、训练标签、验证标签

In [ ]:
model=lgb.LGBMClassifier()
model.fit(x_train,y_train)
y_val_pred=model.predict_proba(x_val)[:,1]

print(f'AUC:{roc_auc_score(y_val,y_val_pred)}')
print(f'logloss:{log_loss(y_val,y_val_pred)}')

y_pred=model.predict_proba(test_df[features])[:,1]
# 训练LightGBM模型并进行预测
# lgb.LGBMClassifier(): 创建LightGBM分类器
# model.fit(): 在训练数据上拟合模型
# predict_proba(): 预测概率，返回每个类别的概率
# [:,1]: 取第二列，即正类（类别1）的概率
# roc_auc_score: 计算AUC指标，评估模型性能
# log_loss: 计算对数损失，另一个评估指标

In [ ]:
import sklearn


sklearn_sub_df=pd.DataFrame({
    'ID':test_df['ID'],
    'label':y_pred
})
sklearn_sub_df.to_csv('/kaggle/working/sklearn_baseline.csv',index=False)
# 创建提交文件
# pd.DataFrame(): 创建数据框，包含ID和预测结果
# 'ID': 测试样本的唯一标识符
# 'label': 模型预测的概率值
# to_csv(): 将结果保存为CSV文件
# '/kaggle/working/sklearn_baseline.csv': 保存路径，符合Kaggle格式
# index=False: 不保存行索引
# 这个文件可以直接提交到Kaggle竞赛平台